#### 1. Library Install & Load / Basic Setting

In [ ]:
!git clone https://github.com/UKPLab/sentence-transformers.git

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 11495, done.
remote: Counting objects: 100% (3614/3614), done.
remote: Compressing objects: 100% (941/941), done.
remote: Total 11495 (delta 3154), reused 2681 (delta 2673), pack-reused 7881 (from 1)
Receiving objects: 100% (11495/11495), 25.93 MiB | 23.90 MiB/s, done.
Resolving deltas: 100% (8180/8180), done.


In [ ]:
!git clone https://github.com/kakaobrain/kor-nlu-datasets.git

Cloning into 'kor-nlu-datasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 16 (delta 1), reused 1 (delta 1), pack-reused 14 (from 1)
Receiving objects: 100% (16/16), 46.67 MiB | 17.79 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip uninstall sentence_transformers

Found existing installation: sentence-transformers 3.3.1
Uninstalling sentence-transformers-3.3.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/sentence_transformers-3.3.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-3.3.1


In [ ]:
%cd sentence-transformers

/content/sentence-transformers


In [ ]:
!pip install -e .

Obtaining file:///content/sentence-transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sentence-transformers (pyproject.toml) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-3.4.0.dev0-0.editable-py3-none-any.whl size=10980 sha256=bffe640d6ca41e87909c4c9bba1f168035f1c95efb15b99b6e36f047c3c698ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-xadd_cz0/wheels/20/79/ec/b839e68c83b7f73dfbf15cf4f989275c8d4eed6b3e247eecbd
Successfully built sentence-transformers


In [ ]:
import sys
import os

# 현재 워킹 디렉토리를 /content로 변경
os.chdir("/content")

# sentence-transformers 폴더의 경로를 sys.path에 추가
sys.path.append("/content/sentence-transformers")

sentence-transformers 모듈을 성공적으로 불러왔습니다!


In [ ]:
import os
import csv
import math
import torch
import random
import logging
import numpy as np

from datasets import Dataset
from datetime import datetime
from torch.utils.data import DataLoader

from sentence_transformers import (
    SentenceTransformer,
    LoggingHandler,
    datasets,
    models,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction
from sentence_transformers.readers import InputExample

In [ ]:
# 로거 설정
log_file = "log_output.txt"

logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[
        logging.FileHandler(log_file),  # 파일 핸들러로 로그를 파일에 기록
        logging.StreamHandler()        # 콘솔에도 출력
    ]
)

# 시드 고정
SEED = 777
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
# 데이터 경로 설정
KORNLI_PATH = '/content/kor-nlu-datasets/KorNLI'
KORSTS_PATH = '/content/kor-nlu-datasets/KorSTS'
OUTPUT_DIR = 'output'

In [ ]:
final_scores = {}

In [ ]:
logging.info("설정 및 라이브러리, 데이터 git 복제 완료.")

#### 2. Data Loader Configure

In [ ]:
# KorSTS 데이터셋을 읽어와 InputExample 리스트를 반환
def load_kor_sts_samples(filename):
    samples = []
    with open(filename, 'rt', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            score = float(row['score']) / 5.0 # Normalize
            samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [ ]:
def load_kor_nli_samples(filename):
    """
    KorNLI 데이터를 읽어와서, Entailment vs Contradiction를 이용해
    MultipleNegativesRankingLoss에 맞는 형태(InputExample)를 생성.
    """
    data = {}
    def add_to_samples(sent1, sent2, label):
        if sent1 not in data:
            data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
        data[sent1][label].add(sent2)

    with open(filename, 'r', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            sent1 = row['sentence1'].strip()
            sent2 = row['sentence2'].strip()
            gold_label = row['gold_label'].strip()
            if gold_label not in ['entailment','contradiction','neutral']:
                # 혹시 라벨이 이상하면 건너뛰기
                continue
            add_to_samples(sent1, sent2, gold_label)
            add_to_samples(sent2, sent1, gold_label)

    samples = []
    for sent1, labels_dict in data.items():
        # entailment/contradiction이 각각 하나 이상 있어야 hard negative 구성 가능
        if len(labels_dict['entailment'])>0 and len(labels_dict['contradiction'])>0:
            pos = random.choice(list(labels_dict['entailment']))
            neg = random.choice(list(labels_dict['contradiction']))
            samples.append(InputExample(texts=[sent1, pos, neg]))

            # sent1을 가운데 두고, pos가 앞뒤로 바뀐 또다른 예시도 추가
            # (Optional - 다양하게 hard negative샘플을 늘릴 수 있음)
            samples.append(InputExample(texts=[pos, sent1, neg]))

    return samples

In [ ]:
logging.info("Data Utilize(Pre-Processing) / Sample Extraction 완료")

In [ ]:
def evaluate_on_sts_test(model_path, batch_size=64):
    """
    주어진 model_path에 있는 모델을 로드하고
    KorSTS Test 세트로 EmbeddingSimilarityEvaluator 시행 후
    최종 스코어를 반환합니다.
    """
    test_file_sts = os.path.join(KORSTS_PATH, "sts-test.tsv")
    test_samples_sts = load_kor_sts_samples(test_file_sts)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
        test_samples_sts,
        batch_size=batch_size,
        name='sts-test'
    )

    model = SentenceTransformer(model_path)
    score = test_evaluator(model)
    return score

#### 3. Experiment 1 - STS only

In [ ]:
##################################################
# 실험 1: STS Only                               #
##################################################
def train_sts_only(
    model_name_or_path="klue/roberta-base",
    max_seq_length=128,
    batch_size=8,
    num_epochs=1,
    output_dir=OUTPUT_DIR
):
    """
    1) STS Only 실험을 진행하고, 모델이 저장된 디렉토리 경로를 반환
    """
    # 결과 저장 경로
    model_save_path = os.path.join(
        output_dir,
        f"sts-only-{model_name_or_path.replace('/', '-')}-{datetime.now().strftime('%m-%d')}"
    )
    logging.info(f"[STS Only] 모델 세이브 경로: {model_save_path}")

    # 모델 구성
    word_embedding_model = models.Transformer(model_name_or_path, max_seq_length=max_seq_length)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False
    )
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    # 데이터 로드
    train_file = os.path.join(KORSTS_PATH, "sts-train.tsv")
    dev_file = os.path.join(KORSTS_PATH, "sts-dev.tsv")

    train_samples = load_kor_sts_samples(train_file)
    dev_samples = load_kor_sts_samples(dev_file)

    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)
    dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=batch_size, name='sts-dev')

    train_loss = losses.CosineSimilarityLoss(model=model)

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
    logging.info(f"STS warmup steps: {warmup_steps}")

    # 학습
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=dev_evaluator,
        epochs=num_epochs,
        evaluation_steps=500,
        warmup_steps=warmup_steps,
        output_path=model_save_path
    )

    logging.info("[STS Only] 학습 완료!")
    return model_save_path

In [ ]:
sts_path = train_sts_only(
        model_name_or_path="klue/roberta-base",
        max_seq_length=128,
        batch_size=16,
        num_epochs=4,
        output_dir=OUTPUT_DIR
    )
# 평가 & 스코어 저장
final_scores['sts_only'] = evaluate_on_sts_test(sts_path, batch_size=8)
logging.info(f"[STS Only] Test Score: {final_scores['sts_only']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.857441,0.857290
500,0.039400,No log,0.857569,0.857545
720,0.039400,No log,0.864980,0.865174
1000,0.014700,No log,0.861506,0.862136
1080,0.014700,No log,0.865067,0.865128
1440,0.014700,No log,0.865847,0.866121


In [ ]:
import shutil

# 압축할 폴더와 생성할 zip 파일 경로 설정
folder_to_zip = "/content/output"
zip_file_path = "/content/output.zip"

# 폴더를 압축
shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_to_zip)
print(f"폴더가 압축되었습니다: {zip_file_path}")

폴더가 압축되었습니다: /content/output.zip


In [ ]:
print(f"[STS Only] Test Score: {final_scores['sts_only']}")

[STS Only] Test Score: {'sts-test_pearson_cosine': 0.8161464939414811, 'sts-test_spearman_cosine': 0.8142776748758852}
